In [1]:
from IPython import display
from matplotlib import pyplot as plt
from mxnet import nd, autograd
import random
%matplotlib inline
display.set_matplotlib_formats("svg")

In [2]:
num_inputs = 2
num_example = 1000
true_w = nd.array([2,-3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_example, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

## reading data

In [4]:
from mxnet.gluon import data as gdata
# combine the features and labels of the training data
batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
# randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

## define the model 

In [5]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

## initialize 

In [6]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

## define the loss function

In [9]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

## define the optimization algorithm

In [11]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {"learning_rate":0.03})

## training

In [12]:
num_epochs = 3
for epoch in range(1, num_epochs+1):
    for x,y in data_iter:
        with autograd.record():
            l=loss(net(x),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print("epoch {}, loss {}".format(epoch, l.mean().asnumpy()))

epoch 1, loss [0.03518308]
epoch 2, loss [0.00013206]
epoch 3, loss [4.8448328e-05]


## evaluate

In [14]:
w = net[0].weight.data()
print("error in w:", true_w-w.reshape(true_w.shape))
b = net[0].bias.data()
print("error in b:", true_b-b)

error in w: 
[-0.00012422 -0.00012755]
<NDArray 2 @cpu(0)>
error in b: 
[-0.00016069]
<NDArray 1 @cpu(0)>
